In [37]:
 from collections import Counter, defaultdict
 from scipy.sparse import csr_matrix
 import math
 from sklearn.preprocessing import normalize
 import numpy as np 

In [2]:
import pandas as pd
import MeCab
tagger = MeCab.Tagger("-Owakati")

In [3]:
df = pd.read_csv('/home/ifte/Downloads/chat_history.csv')

In [4]:
df.head()

,query_cleaning
0,明日の天気
1,今日の天気
2,ディズニー
3,今日の天気
4,今日の天気


In [5]:
collector = []
for items in df.query_cleaning.values:
    collector.append(tagger.parse(items))

In [7]:
gen = pd.DataFrame(zip(collector, df.query_cleaning.values.tolist()), columns=['pro', 'raw'])

In [36]:
gen['raw'].iloc[0]

'明日の天気'

In [11]:
gen.head()

,pro,raw
0,明日 の 天気 \n,明日の天気
1,今日 の 天気 \n,今日の天気
2,ディズニー \n,ディズニー
3,今日 の 天気 \n,今日の天気
4,今日 の 天気 \n,今日の天気


In [30]:
 def IDF(corpus, unique_words):
   idf_dict={}
   N=len(corpus)
   for i in unique_words:
     count=0
     for sen in corpus:
       if i in sen.split():
         count=count+1
       idf_dict[i]=(math.log((1+N)/(count+1)))+1
   return idf_dict 

 def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
      for x in whole_data:
        for y in x.split():
          if len(y)<2:
            continue
          unique_words.add(y)
      unique_words = sorted(list(unique_words))
      vocab = {j:i for i,j in enumerate(unique_words)}
      Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words

def transform(dataset,vocabulary,idf_values):
     sparse_matrix= csr_matrix( (len(dataset), len(vocabulary)), dtype=np.float64)
     for row  in range(0,len(dataset)):
       number_of_words_in_sentence=Counter(dataset[row].split())
       for word in dataset[row].split():
           if word in  list(vocabulary.keys()):
               tf_idf_value=(number_of_words_in_sentence[word]/len(dataset[row].split()))*(idf_values[word])
               sparse_matrix[row,vocabulary[word]]=tf_idf_value     
#      output =normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
     return normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)

In [12]:
corpus = gen.pro.values.tolist()

In [31]:
Vocabulary, idf_of_vocabulary=fit(corpus)
final_output=transform(corpus,Vocabulary,idf_of_vocabulary)

/home/ifte/environments/experiment_env/lib/python3.6/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [34]:
final_output.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
len(gen)

5130

In [59]:
s = SampleSegregate(k=int(len(gen)/25))

In [60]:
out = s.fit(final_output.toarray(), gen)

/home/ifte/environments/experiment_env/lib/python3.6/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


In [61]:
d = pd.DataFrame(out)

In [62]:
d.columns

Index(['class', 'raw', 'pro'], dtype='object')

In [63]:
d.sort_values(by='class')

,class,raw,pro
0,0,明日の天気,明日 の 天気 \n
870,0,明日の天気,明日 の 天気 \n
867,0,明日の天気,明日 の 天気 \n
1329,0,明日の天気,明日 の 天気 \n
254,0,明日の天気,明日 の 天気 \n
...,...,...,...
203,202,時間,時間 \n
4315,202,時間 返せる,時間 返せる \n
216,204,時間 説明,時間 説明 \n
204,204,時間 説明,時間 説明 \n


In [ ]:
s.classes

In [30]:
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
import MeCab
import math
import re

tagger = MeCab.Tagger("-Owakati")


def inv_dict_gen(corpus, unique_words):
    inv_dict = {}
    for i in unique_words:
        count = 0
        for sen in corpus:
            if i in sen.split():
                count = count + 1
            inv_dict[i] = (math.log((1 + len(corpus)) / (count + 1))) + 1
    return inv_dict


def train_data(dataset):
    unique_words = set()
    for x in dataset:
        for y in x.split():
            if len(y) < 3 and bool(re.compile('[a-z]').match(y)):
                continue
            unique_words.add(y)

    unique_words = sorted(list(unique_words))
    vocabulary = {j: i for i, j in enumerate(unique_words)}
    inv_values = inv_dict_gen(dataset, unique_words)
    sparse_matrix = csr_matrix((len(dataset), len(vocabulary)), dtype=np.float64)

    for row in range(0, len(dataset)):
        words = Counter(dataset[row].split())
        for word in dataset[row].split():
            if word in list(vocabulary.keys()):
                sparse_matrix[row, vocabulary[word]] = (words[word] / len(dataset[row].split())) * (inv_values[word])

    return normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False).toarray()


# Segregate the samples
class SampleSegregate:
    def __init__(self, k=3, tolerance=0.0001, max_iterations=500):
        self.classes = {}
        self.centroids = {}
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations

    def fit_matrix(self, data, dataframe):
        saver = {}
        self.centroids = {}
        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iterations):
            self.classes = {}
            for i in range(self.k):
                self.classes[i] = []

            # distance [point <=> cluster]
            c = 0
            saver = defaultdict(list)
            for features in data:
                distances = [np.linalg.norm(features - self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classes[classification].append(features)
                saver['cluster'].append(classification)
                saver['text'].append(dataframe['raw'].iloc[c])
                c += 1

            previous = dict(self.centroids)
            # average cluster data points re-calculate centroids
            for classification in self.classes:
                self.centroids[classification] = np.average(self.classes[classification], axis=0)

            optimal = True
            for centroid in self.centroids:
                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                if np.sum((curr - original_centroid) / original_centroid * 100.0) > self.tolerance:
                    optimal = False

            # break if optimal
            if optimal:
                break

        return pd.DataFrame(saver)

    def predict(self, data):
        distances = [np.linalg.norm(data - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification





In [31]:
df = pd.read_csv('/home/ifte/Downloads/chat_history.csv')

collector = []
for items in df.query_cleaning.values:
    collector.append(tagger.parse(items))

gen = pd.DataFrame(zip(collector, df.query_cleaning.values.tolist()), columns=['pro', 'raw'])
matrix = train_data(gen.pro.values.tolist())

/home/ifte/environments/experiment_env/lib/python3.6/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [32]:
s = SampleSegregate(k=10)
# s.fit_matrix(matrix, gen)

In [ ]:
sil_score_max = -1
for n_clusters in range(4, 30):
    labels = s.predict(matrix[:100])
    sil_score = silhouette_score(matrix[:100], labels)
    print("The average silhouette score for %i clusters is %0.2f" % (n_clusters, sil_score))
    if sil_score > sil_score_max:
        sil_score_max = sil_score
        best_n_clusters = n_clusters

In [43]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [39]:
kMeansCluster = KMeans(n_clusters=20)
kMeansCluster.fit(matrix)
clusters = kMeansCluster.labels_.tolist()

In [45]:
km_silhouette = []
for i in range(3,20):
    km = KMeans(n_clusters=i, random_state=0).fit(matrix)
    preds = km.predict(matrix)
    
    silhouette = silhouette_score(matrix, preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))


Silhouette score for number of cluster(s) 3: 0.06252950918587859
Silhouette score for number of cluster(s) 4: 0.07808636490420136
Silhouette score for number of cluster(s) 5: 0.06521017681591557
Silhouette score for number of cluster(s) 6: 0.08817938896485673
Silhouette score for number of cluster(s) 7: 0.08652936533682594
Silhouette score for number of cluster(s) 8: 0.10476718351939034
Silhouette score for number of cluster(s) 9: 0.11170184334180273
Silhouette score for number of cluster(s) 10: 0.12142940097261096
Silhouette score for number of cluster(s) 11: 0.1369359522332204
Silhouette score for number of cluster(s) 12: 0.13714500546961503
Silhouette score for number of cluster(s) 13: 0.14400596793182407
Silhouette score for number of cluster(s) 14: 0.14936710468203
Silhouette score for number of cluster(s) 15: 0.1646809550175829
Silhouette score for number of cluster(s) 16: 0.1718625543645005
Silhouette score for number of cluster(s) 17: 0.16298560760663003
Silhouette score for nu

In [46]:
result = s.fit_matrix(matrix, gen)

/home/ifte/environments/experiment_env/lib/python3.6/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: divide by zero encountered in true_divide
/home/ifte/environments/experiment_env/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


In [47]:
result.cluster.unique()

array([0, 1, 2, 7])

In [35]:
result.text = result.text.apply(lambda x: x+' | ')

In [50]:
result.head()

,cluster,text
0,0,明日の天気
1,1,今日の天気
2,2,ディズニー
3,1,今日の天気
4,1,今日の天気


In [51]:
grouped = result.groupby('cluster', sort=False).sum()
grouped.sort_index(ascending=False)
print(grouped)

                                                      text
cluster                                                   
0        明日の天気記載 ヘルプ 使い方学習 ヘルプami e学習 url なん学習 ファイル追加 学...
1        今日の天気今日の天気今日の天気今日の天気今日の天気今日の天気今日の天気今日の天気今日の天気 ...
2        ディズニーios リモート 操作回答 分類保険料 控除足りる ない た足りる ない た足りる...
7        シナリオ 登録シナリオ 登録シナリオ 登録シナリオ 登録シナリオ 登録シナリオ 登録シナリオ...


In [54]:
result.groupby('cluster').agg(['count'])

,text
,count
cluster,
0,4748
1,155
2,90
7,137


In [ ]:
class            text [0         5           明日の天気] [1         5           今日の天気] [2         5           ディズニー] [3         5           今日の天気] [4         5           今日の天気] [...     ...             ...] [5125      5          スマホワーク] [5126      5  申し込む お ある 

In [145]:
df = pd.read_csv('/home/ifte/Downloads/clustered.csv')

In [147]:
df = df.groupby('class').agg(['count'])
# sentence_count.columns = ['class', 'Text']
# sentence_count = sentence_count.sort_values(by='Text', ascending=False)

In [ ]:
df.sort_values(by=df.columns[0], ascending=False)

In [ ]:
result.groupby('cluster').agg(['count'])

In [ ]:
sentence_count = df.set_index(["class", "text"]).count(level="class")

sentence_count.columns = ['Text']

sentence_count = sentence_count.reset_index()

sentence_count = sentence_count.sort_values(by='Text', ascending=False)

sentence_count = sentence_count.reset_index()

sentence_count.columns = ['cluster', 'Text']

sentence_count = sentence_count.sort_values(by='Text', ascending=False)

In [ ]:
sentence_count

In [ ]:
df[(df['class'] == 1)].text.values.tolist()

In [85]:
sentence_count = df.set_index(["class", "text"]).count(level="class")
sentence_count = sentence_count.reset_index()
sentence_count.columns = ['cluster', 'Text']
sentence_count.sort_values(by='Text', ascending=False)

In [80]:
grouped = df.groupby('class')[['text']].agg(['count'])
grouped = grouped.reset_index()

In [74]:
grouped.columns

Index(['count'], dtype='object')

In [70]:
grouped.columns = ['class', 'Text']

In [81]:
grouped.head()

,class,text
,,count
0,0,87
1,1,100
2,2,170
3,3,322
4,4,65
